In [ ]:
import pandas as pd
import importlib
import parse_chess_state as parsed
importlib.reload(parsed)

datasets = pd.read_csv('./datasets/check/many_pieces.txt', sep=' ', header=None)
data = parsed.parallel_batch_fens_to_arrays(datasets[0][:10000].values.tolist(), 4)
parsed.print_board(data[0])


Current board state:
 4  0  4  0  0  0  6  0
 0  0 -2  2  0  1  2  0
 1  0  0  1  1 -3  1  0
 0 -1  0  0 -1  0  0  0
 0  0  0  0  0  0  0  5
 0 -1  0  0  0  0  0  0
-1  0  0 -5 -2  0 -1 -1
 0  0 -4  0  0 -4 -6  0



In [32]:
data.shape

(10000, 8, 8)

Current board state:
 4  0  4  0  0  0  6  0
 0  0 -2  2  0  1  2  0
 1  0  0  1  1 -3  1  0
 0 -1  0  0 -1  0  0  0
 0  0  0  0  0  0  0  5
 0 -1  0  0  0  0  0  0
-1  0  0 -5 -2  0 -1 -1
 0  0 -4  0  0 -4 -6  0

